In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string
import docx

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental import preprocessing

In [270]:
csv_ds = tf.data.experimental.make_csv_dataset(
    'csv/train.csv',
    batch_size=1, # Artificially small to make examples easier to show.
    label_name='spam',
    num_epochs=1,
    ignore_errors=True)

In [271]:
train_ds = csv_ds.take(1200)
val_ds = csv_ds.skip(1200)

In [272]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [273]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [274]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

In [275]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x['email'])
vectorize_layer.adapt(text_ds)

In [276]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [277]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [278]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [279]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[tensorboard_callback])

Epoch 1/5
1200/1200 [==============================] - 10s 8ms/step - loss: 0.4170 - accuracy: 0.7633 - val_loss: 0.2639 - val_accuracy: 0.7764
Epoch 2/5
1200/1200 [==============================] - 8s 7ms/step - loss: 0.1510 - accuracy: 0.9383 - val_loss: 0.1247 - val_accuracy: 0.9477
Epoch 3/5
1200/1200 [==============================] - 9s 7ms/step - loss: 0.0647 - accuracy: 0.9792 - val_loss: 0.0881 - val_accuracy: 0.9639
Epoch 4/5
1200/1200 [==============================] - 8s 7ms/step - loss: 0.0340 - accuracy: 0.9883 - val_loss: 0.0790 - val_accuracy: 0.9695
Epoch 5/5
1200/1200 [==============================] - 8s 7ms/step - loss: 0.0182 - accuracy: 0.9967 - val_loss: 0.0797 - val_accuracy: 0.9715


In [ ]:
model.summary()

In [147]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

470/470 [==============================] - 1s 2ms/step - loss: 0.0183 - accuracy: 0.9970
Loss:  0.0182857196778059
Accuracy:  0.9970187544822693


array([[0.00742621]], dtype=float32)